In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from merge.modules.config import TransformerConfig
from merge.modules.transformer import Transformer


In [ ]:
from merge.preprocessing.tokenizer import AutoTokenizer


tokenizer = AutoTokenizer.create("bert-base-uncased")

In [ ]:
tokenizer.encode("Hello, my dog is cute")

In [ ]:
from transformers import AutoTokenizer as HFAutoTokenizer


class HFTokenizerWrapper:
    """Wrapper for HuggingFace tokenizers that implements our interface"""

    def __init__(self, name: str):
        super().__init__()
        self.tokenizer = HFAutoTokenizer.from_pretrained(name)

    def __getattr__(self, name):
        return getattr(self.tokenizer, name)
    
    def __call__(self, *args, **kwargs):
        # Delegate the call to the underlying tokenizer
        return self.tokenizer(*args, **kwargs)

In [ ]:
tokenizer = HFTokenizerWrapper("bert-base-uncased")

In [ ]:
text = "This is a test"
tokenizer(text, 2)

In [ ]:
hftokenizer = HFAutoTokenizer.from_pretrained("bert-base-uncased")
text = "This is a test"
hftokenizer(text)

In [ ]:
from datasets import load_dataset

In [ ]:
# use name="sample-10BT" to use the 10BT sample
fw = load_dataset("HuggingFaceFW/fineweb-edu", name="sample-10BT", split="train", cache_dir="./hf_cache", num_proc=8)


In [ ]:
fw = load_dataset("roneneldan/TinyStories", split="train", cache_dir="./hf_cache", num_proc=8)


In [ ]:
from merge.preprocessing.tokenizer import AutoTokenizer

tokenizer = AutoTokenizer.create("EleutherAI/gpt-neo-125m")

In [ ]:
max_seq_length = 1024

fw_test = fw.map(
    lambda x: tokenizer(
        x["text"],
        truncation=True,
        max_length=max_seq_length,
        padding="max_length"
    ),
    batched=True,
    remove_columns=["text"],
    num
)

In [ ]:
fw_tokenized = _tokenize(fw, tokenizer, 1)

In [ ]:
process_sequences(fw_tokenized, 256, 1024, 16)

In [ ]:
import torch
ones = torch.ones((1, 50))